## Working with DNA sequence data for machine learning 
source: https://www.kaggle.com/thomasnelson/working-with-dna-sequence-data-for-ml

- Python package Biopython: tools for biological computation.
- fasta: a file that contains DNA sequence data 

In [ ]:
# !pip install Bio

In [3]:
from Bio import SeqIO
for seq_record in SeqIO.parse('./data/example.fa', 'fasta'):
  print(seq_record.id)
  print(seq_record.seq)

ENST00000435737.5
ATGTTTCGCATCACCAACATTGAGTTTCTTCCCGAATACCGACAAAAGGAGTCCAGGGAATTTCTTTCAGTGTCACGGACTGTGCAGCAAGTGATAAACCTGGTTTATACAACATCTGCCTTCTCCAAATTTTATGAGCAGTCTGTTGTTGCAGATGTCAGCAACAACAAAGGCGGCCTCCTTGTCCACTTTTGGATTGTTTTTGTCATGCCACGTGCCAAAGGCCACATCTTCTGTGAAGACTGTGTTGCCGCCATCTTGAAGGACTCCATCCAGACAAGCATCATAAACCGGACCTCTGTGGGGAGCTTGCAGGGACTGGCTGTGGACATGGACTCTGTGGTACTAAATGAAGTCCTGGGGCTGACTCTCATTGTCTGGATTGACTGA
ENST00000419127.5
ATGTTTCGCATCACCAACATTGAGTTTCTTCCCGAATACCGACAAAAGGAGTCCAGGGAATTTCTTTCAGTGTCACGGACTGTGCAGCAAGTGATAAACCTGGTTTATACAACATCTGCCTTCTCCAAATTTTATGAGCAGTCTGTTGTTGCAGATGTCAGCAACAACAAAGGCGGCCTCCTTGTCCACTTTTGGATTGTTTTTGTCATGCCACGTGCCAAAGGCCACATCTTCTGTGAAGACTGTGTTGCCGCCATCTTGAAGGACTCCATCCAGACAAGCATCATAAACCGGACCTCTGTGGGGAGCTTGCAGGGACTGGCTGTGGACATGGACTCTGTGGTACTAAATGACAAAGGCTGCTCTCAGTACTTCTATGCAGAGCATCTGTCTCTCCACTACCCGCTGGAGATTTCTGCAGCCTCAGGGAGGCTGATGTGTCACTTCAAGCTGGTGGCCATAGTGGGCTACCTGATTCGTCTCTCAATCAAGTCCATCCAAATCGAAGCCGACAACTGTGTCACTGACTCCCTGACCATTTACGACTCCCTTTTGCCCATCCGGAGCAGCATC

### Apply ML & deep learning 
1. Encode the sequence information as an ordinal vector
2. One-hot encode the sequence letters and use the resulting array
3. Treat the DNA sequence as __a language__ and use various language processing methods 

### 1. Ordinal encoding DNA sequence data 
First, create some utility functions such as for creating a numpy array object from a sequence string, and a label encoder with the DNA sequence alphabel 'a', 'c', 'g' and 't', but also a character for anything else, 'n'

In [5]:
"""
function to convert a DNA sequence string to a numpy array
convers to lower case, changes any non 'acgt' character to 'n'
"""
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder

def string_to_array(my_string):
  my_string = my_string.lower()
  my_string = re.sub('[^acgt]', 'z', my_string)
  my_array = np.array(list(my_string))
  return my_array

# create a label encoder with 'acgtn' alphabet 
label_enc = LabelEncoder()
label_enc.fit(np.array(['a','c','g','t','z']))

LabelEncoder()

In [6]:
"""
function to encode a DNA sequence string as an ordinal vector 
returns a numpy vector with a=0.25, c=0.50, g=0.75, t=1.00, n=0.00
"""
def ordinal_encoder(arr):
  int_encoded = label_enc.transform(arr)
  float_encoded = int_encoded.astype(float)
  float_encoded[float_encoded == 0] = 0.25 # A
  float_encoded[float_encoded == 1] = 0.50 # C
  float_encoded[float_encoded == 2] = 0.75 # G
  float_encoded[float_encoded == 3] = 1.00 # T
  float_encoded[float_encoded == 4] = 0.00 # anything else, z
  return float_encoded

In [8]:
sample_seq = "AACGCGCTTNN"
ordinal_encoder(string_to_array(sample_seq))

array([0.25, 0.25, 0.5 , 0.75, 0.5 , 0.75, 0.5 , 1.  , 1.  , 0.  , 0.  ])

### 2. One-hot encoding DNA sequence data 
widely used in deep learning methods and lends itself well to algorithms like convolutional neural network. 

In [15]:
"""
function to one-hot encode a DNA sequence string 
non 'acgt' bases (n) are 0000
returns a L x 4 numpy array
"""
from sklearn.preprocessing import OneHotEncoder

def one_hot_encoder(my_array):
    integer_encoded = label_enc.transform(my_array)
    onehot_encoder = OneHotEncoder(sparse=False, dtype=int)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    onehot_encoded = np.delete(onehot_encoded, -1, 1)
    return onehot_encoded

In [16]:
one_hot_encoder(string_to_array(sample_seq))

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

### 3. Treating DNA sequence as an "language", otherwise known as k-mer counting
- Take the long biological sequence and break it down into k-mer length overlapping words.
- Word length can be tuned to suit the particular situation. The word length and amount of overlap need to be determined impirically for any given application 
- Refer to these types of manipulations as 'k-mer counting'

In [17]:
def getKmers(sequence, size):
  return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

In [19]:
sample_seq = 'CATGGCCATCCCCCCCCGAGCGGGGGGGGGG'
getKmers(sample_seq, size=6)

['catggc',
 'atggcc',
 'tggcca',
 'ggccat',
 'gccatc',
 'ccatcc',
 'catccc',
 'atcccc',
 'tccccc',
 'cccccc',
 'cccccc',
 'cccccc',
 'cccccg',
 'ccccga',
 'cccgag',
 'ccgagc',
 'cgagcg',
 'gagcgg',
 'agcggg',
 'gcgggg',
 'cggggg',
 'gggggg',
 'gggggg',
 'gggggg',
 'gggggg',
 'gggggg']

The above function call returns a list of k-mer __words__. You can join the words into a sentence, then apply NLP methods on the sentences as you normally would.   
- Can tune both the word length and the amount of overlap. This allows you to determine how sequence information and vocabulary size will be important in your application. 
- For example, if you use words of length 6, and there are 4 letters, you have a vocabulary of size 4096 possible words. You can go on and create a bag-of-words model in NLP.

In [20]:
words = getKmers(sample_seq, size=6)
sentence = ' '.join(words)
sentence

'catggc atggcc tggcca ggccat gccatc ccatcc catccc atcccc tccccc cccccc cccccc cccccc cccccg ccccga cccgag ccgagc cgagcg gagcgg agcggg gcgggg cggggg gggggg gggggg gggggg gggggg gggggg'

In [21]:
seq1 = 'GATGGCCATCCCCGCCCGAGCGGGGGGGG'
seq2 = 'CATGGCCATCCCCGCCCGAGCGGGCGGGG'
sentence1 = ' '.join(getKmers(seq1, size=6))
sentence2 = ' '.join(getKmers(seq2, size=6))

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
# bag of words model
cv = CountVectorizer()
X = cv.fit_transform([sentence, sentence1, sentence2]).toarray()

In [24]:
X

array([[1, 1, 1, 1, 1, 1, 3, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,
        0, 1, 1, 0, 0, 5, 1, 0, 1],
       [1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        0, 1, 1, 0, 0, 3, 0, 1, 1],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 0, 0, 1, 1]])